In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, SparseVectorParams, Document, Prefetch, FusionQuery, PointStruct
from qdrant_client.http import models

import pandas as pd
import openai

import fastembed


import cohere

/Users/tanyakanodia/learning-ai/00-ai-engineering-bootcamp-cohort-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [17]:
cohere_client = cohere.ClientV2()

In [7]:
def get_embeddings(text, model="text-embedding-3-small"):
    """
    Returns embeddings for the given text using OpenAI model 'text-embedding-3-small'.
    """
    response = openai.embeddings.create(
        model=model,
        input=[text]
    )
    # Assuming response shape matches openai API: response.data[0].embedding
    return response.data[0].embedding


In [8]:

def retrieve_data(query, qdrant_client, k=10):
    """
    Retrieve k most similar items to the query from Qdrant collection.
    """
    query_embedding = get_embeddings(query)

    response = qdrant_client.query_points(
        collection_name="Amazon-items-collection-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k
    )

    retrieved_context_ids = [];
    # this is description of the product
    retrieved_context = [];
    similarity_scores = [];
    retrieved_content_ratings = []
    for point in response.points:   
        retrieved_context_ids.append(point.payload["parent_asin"])
        retrieved_context.append(point.payload["description"])
        similarity_scores.append(point.score)
        retrieved_content_ratings.append(point.payload["average_rating"])
    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
        "retrieved_content_ratings": retrieved_content_ratings
    }


In [12]:
query = "Can I get some tablet?"

In [13]:
result = retrieve_data(query, qdrant_client, k=10)   

In [14]:
result

{'retrieved_context_ids': ['B0B8NVNQKX',
  'B09LH466KZ',
  'B0BR33XH8D',
  'B0BN7WWH63',
  'B0C78B1BTB',
  'B0BL2CZSHT',
  'B0C3XYD574',
  'B0BFGXRMJN',
  'B0C772GZRF',
  'B08CGM6Y1J'],
 'retrieved_context': ['COOPERS 7 inch Kids Tablet Android 11 Tablet for Kids, 2GB RAM + 32GB ROM Toddler Tablet PC for Children, IPS Touch Screen, Dual Camera, Dual Speaker, WiFi Computer Tablet, Light Blue ✿【Good Kids tablet】This 7 inch tablet for kids with silm body and lightweight, it is easy to hold by children. Also the special design can protect the tablet well when dropping.✿【Parental Control】Toddlers Tablet with parent mode can add or block apps. Set screen time limits. This tablet come with iwawa app. kids can get access to fun and educational games and videos.✿【Powerful Tablets】Equipmented with quad core CPU, Android 11.0 System, 32GB Big Storage, 1024*600 IPS Screen offer a clear view. Runs Kinds of apps and game for kids smoothly.✿【Long Lasting】Tablet for kids built with large capacity batt

In [15]:
to_rerank = result["retrieved_context"]

In [19]:
response = cohere_client.rerank(
    model="rerank-v3.5",
    query=query,
    documents=to_rerank,
    top_n=20
)


In [20]:
response

V2RerankResponse(id='f7c29a9c-d9c8-4b49-a09c-a5977567fc65', results=[V2RerankResponseResultsItem(index=0, relevance_score=0.17613278), V2RerankResponseResultsItem(index=6, relevance_score=0.17401722), V2RerankResponseResultsItem(index=4, relevance_score=0.1632989), V2RerankResponseResultsItem(index=1, relevance_score=0.16265947), V2RerankResponseResultsItem(index=5, relevance_score=0.1479859), V2RerankResponseResultsItem(index=8, relevance_score=0.1436799), V2RerankResponseResultsItem(index=3, relevance_score=0.13828741), V2RerankResponseResultsItem(index=7, relevance_score=0.13333645), V2RerankResponseResultsItem(index=9, relevance_score=0.103939675), V2RerankResponseResultsItem(index=2, relevance_score=0.10071066)], meta=ApiMeta(api_version=ApiMetaApiVersion(version='2', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(images=None, input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, cached_tokens=None, warnings=None))

In [23]:
response.results

[V2RerankResponseResultsItem(index=0, relevance_score=0.17613278),
 V2RerankResponseResultsItem(index=6, relevance_score=0.17401722),
 V2RerankResponseResultsItem(index=4, relevance_score=0.1632989),
 V2RerankResponseResultsItem(index=1, relevance_score=0.16265947),
 V2RerankResponseResultsItem(index=5, relevance_score=0.1479859),
 V2RerankResponseResultsItem(index=8, relevance_score=0.1436799),
 V2RerankResponseResultsItem(index=3, relevance_score=0.13828741),
 V2RerankResponseResultsItem(index=7, relevance_score=0.13333645),
 V2RerankResponseResultsItem(index=9, relevance_score=0.103939675),
 V2RerankResponseResultsItem(index=2, relevance_score=0.10071066)]

In [ ]:
# This list comprehension creates a new list 'reranked_results' by reordering 'to_rerank'
# according to the ranking specified by 'response.results'.
# For each 'results' object in 'response.results', it retrieves the index (results.index)
# which points to the original position in 'to_rerank', and collects the corresponding document.
# basically it reassigns orders or indexes to the list, ex: in the response.results item 1 is the index 2 of to_rerank - then with this it will become item 0 index.
reranked_results = [to_rerank[results.index] for results in response.results]
reranked_results

['COOPERS 7 inch Kids Tablet Android 11 Tablet for Kids, 2GB RAM + 32GB ROM Toddler Tablet PC for Children, IPS Touch Screen, Dual Camera, Dual Speaker, WiFi Computer Tablet, Light Blue ✿【Good Kids tablet】This 7 inch tablet for kids with silm body and lightweight, it is easy to hold by children. Also the special design can protect the tablet well when dropping.✿【Parental Control】Toddlers Tablet with parent mode can add or block apps. Set screen time limits. This tablet come with iwawa app. kids can get access to fun and educational games and videos.✿【Powerful Tablets】Equipmented with quad core CPU, Android 11.0 System, 32GB Big Storage, 1024*600 IPS Screen offer a clear view. Runs Kinds of apps and game for kids smoothly.✿【Long Lasting】Tablet for kids built with large capacity battery. For mixed use up to 6 hours. You can enjoy happy parent-child time with your kids.✿【Best Gift】This high performance Children tablet is designed specially for kids. Best gift choice for Christmas, New Yea